stations_with_descriptionsThe manual: http://data.atoc.org/sites/all/themes/atoc/files/RSPS5045.pdf

The data: http://data.atoc.org/fares-data

In [4]:
import pandas as pd
import sys
sys.path.append("./..")
from importlib import reload

from src import data_prep
import config.indices_of_records as dicts

In [5]:
reload(dicts)
files_pattern = "RJFAF689"

### Flow file

In [129]:
reload(data_prep)

<module 'src.data_prep' from './..\\src\\data_prep.py'>

In [130]:
merged_data = data_prep.prep_fares_table(files_pattern=files_pattern)

In [131]:
merged_data.head()

,records_x,update_marker,record_type_x,origin_code,destination_code,route_code,status_code,usage_code,direction,end_date,...,toc,cross_london_ind,non_standard_disc_ind,publication_ind,flow_id,records_y,record_type_y,ticket_code,fare,restriction_code
0,RF0027003201000000AS3112299902012020ATO01Y0000049,R,F,0027,0032,01000,000,A,S,31122999,...,ATO,0,1,Y,0000049,RT00000497TF00005420,T,7TF,00005420,
1,RF0027003201000000AS3112299902012020ATO01Y0000049,R,F,0027,0032,01000,000,A,S,31122999,...,ATO,0,1,Y,0000049,RT00000497TS00003610,T,7TS,00003610,
2,RF0027003301000000AS3112299902012020ATO01Y0000028,R,F,0027,0033,01000,000,A,S,31122999,...,ATO,0,1,Y,0000028,RT00000287TF00006360,T,7TF,00006360,
3,RF0027003301000000AS3112299902012020ATO01Y0000028,R,F,0027,0033,01000,000,A,S,31122999,...,ATO,0,1,Y,0000028,RT00000287TS00004240,T,7TS,00004240,
4,RF0027003401000000AS3112299902012020ATO01Y0000165,R,F,0027,0034,01000,000,A,S,31122999,...,ATO,0,1,Y,0000165,RT00001657TF00007790,T,7TF,00007790,


## Locations file
Locations, including group locations

In [125]:
locations_records, synonym_records = data_prep.prep_locations_tables(files_pattern=files_pattern)

In [136]:
reload(data_prep)

<module 'src.data_prep' from './..\\src\\data_prep.py'>

### Routes

In [137]:
routes_records, route_excl_records = data_prep.prep_routes_tables(files_pattern=files_pattern)

In [138]:
routes_records.head()

,records,update_marker,record_type,route_code,end_date,start_date,quote_date,description
1215,RR00000311229990303201807042017ANY PERMITTED ...,R,R,00000,31122999,03032018,07042017,ANY PERMITTED
1216,RR00002311229992306201107042017NOT OTHER ZONES...,R,R,00002,31122999,23062011,07042017,NOT OTHER ZONES
1217,RR00003060520200410201806052020STRATFORD/LONDO...,R,R,00003,06052020,04102018,06052020,STRATFORD/LONDON
1218,RR00003311229990705202006052020STRATFORD/LONDO...,R,R,00003,31122999,07052020,06052020,STRATFORD/LONDON
1219,RR00004311229992212201821122018VIA BLACKHORS R...,R,R,00004,31122999,22122018,21122018,VIA BLACKHORS RD


### Combining the data sets

In [142]:
reload(data_prep)

<module 'src.data_prep' from './..\\src\\data_prep.py'>

In [143]:
routes_with_fares_filtered = data_prep.combine_fares_and_routes(
    merged_data=merged_data, routes_records=routes_records)

In [144]:
routes_fares_descriptions = data_prep.combine_fares_routes_locations(
    routes_with_fares_filtered=routes_with_fares_filtered,
    locations_records=locations_records)

In [145]:
routes_fares_descriptions.head()

,update_marker,origin_code,destination_code,route_code,status_code,usage_code,direction,toc,cross_london_ind,non_standard_disc_ind,...,flow_id,ticket_code,fare,restriction_code,quote_date,route_description,nlc_code_origin,origin_station,nlc_code_destination,description
0,R,0027,0032,01000,000,A,S,ATO,0,1,...,0000049,7TF,00005420,,22062020,.,0027,TRVCARD 1,0032,LONDON ZONES 1-2
1,R,0027,0032,01000,000,A,S,ATO,0,1,...,0000049,7TF,00005420,,22062020,.,0027,TRVCARD 1,0032,LONDON ZONES 1-2
2,R,0027,0032,01000,000,A,S,ATO,0,1,...,0000049,7TS,00003610,,22062020,.,0027,TRVCARD 1,0032,LONDON ZONES 1-2
3,R,0027,0032,01000,000,A,S,ATO,0,1,...,0000049,7TS,00003610,,22062020,.,0027,TRVCARD 1,0032,LONDON ZONES 1-2
4,R,0028,0032,01000,000,A,S,ATO,0,1,...,0000226,7TF,00005420,,22062020,.,0028,TRVCARD 2,0032,LONDON ZONES 1-2


In [146]:
routes_fares_descriptions.to_csv("../data/prepped_data.csv")

In [55]:
reload(data_prep)

<module 'src.data_prep' from './..\\src\\data_prep.py'>

### Ticket types

In [56]:
ticket_types = data_prep.prep_ticket_types_table(files_pattern=files_pattern, dictionary_types=dicts.dict_ticket_types)

In [57]:
ticket_types.head()

,records,update_marker,ticket_code,end_date,start_date,quote_date,description,tkt_class,tkt_type,tkt_group,...,max_children,min_children,restricted_by_date,restricted_by_train,restricted_by_area,validity_code,atb_description,lul_xlondon_issue,fare_multiplier,discount_category
0,R0AA311229991707201916072019SMART TKS 2SS...,R,0AA,31122999,17072019,16072019,SMART TKS,2,S,S,...,001,000,N,N,Y,87,SMART TKS,1,001,20
1,R0AB311229991707201916072019SMART TKR 2RS...,R,0AB,31122999,17072019,16072019,SMART TKR,2,R,S,...,001,000,N,N,Y,84,SMART TKR,2,001,20
2,R0AC311229992805201725052017SMART FDR 1RF...,R,0AC,31122999,28052017,25052017,SMART FDR,1,R,F,...,001,000,N,N,N,81,SMART FDR,2,001,01
3,R0AD311229992805201725052017SMART FDS 1SF...,R,0AD,31122999,28052017,25052017,SMART FDS,1,S,F,...,001,000,N,N,N,87,SMART FDS,1,001,01
4,R0AE311229992805201725052017SMART SDR 2RS...,R,0AE,31122999,28052017,25052017,SMART SDR,2,R,S,...,001,000,N,N,N,81,SMART SDR,2,001,01


### Ticket validity

In [42]:
ticket_validity = pd.read_table(
    f"../data/{files_pattern}/{files_pattern}.TVL",
    skipfooter=1, engine="python", names=["records"], header=5)

In [44]:
reload(data_prep)

<module 'src.data_prep' from './..\\src\\data_prep.py'>

In [45]:
ticket_types

,records,update_marker,ticket_code,end_date,start_date,quote_date,description,tkt_class,tkt_type,tkt_group,...,max_children,min_children,restricted_by_date,restricted_by_train,restricted_by_area,validity_code,atb_description,lul_xlondon_issue,fare_multiplier,discount_category
0,R0AA311229991707201916072019SMART TKS 2SS...,R,0AA,31122999,17072019,16072019,SMART TKS,2,S,S,...,001,000,N,N,Y,87,SMART TKS,1,001,20
1,R0AB311229991707201916072019SMART TKR 2RS...,R,0AB,31122999,17072019,16072019,SMART TKR,2,R,S,...,001,000,N,N,Y,84,SMART TKR,2,001,20
2,R0AC311229992805201725052017SMART FDR 1RF...,R,0AC,31122999,28052017,25052017,SMART FDR,1,R,F,...,001,000,N,N,N,81,SMART FDR,2,001,01
3,R0AD311229992805201725052017SMART FDS 1SF...,R,0AD,31122999,28052017,25052017,SMART FDS,1,S,F,...,001,000,N,N,N,87,SMART FDS,1,001,01
4,R0AE311229992805201725052017SMART SDR 2RS...,R,0AE,31122999,28052017,25052017,SMART SDR,2,R,S,...,001,000,N,N,N,81,SMART SDR,2,001,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2823,RZZV311229992305201722052017NOT FOR TRAVEL 9SE...,R,ZZV,31122999,23052017,22052017,NOT FOR TRAVEL,9,S,E,...,000,000,N,N,N,87,NOT FOR TRAVEL,1,001,20
2824,RZZW311229992305201722052017NOT FOR TRAVEL 1SE...,R,ZZW,31122999,23052017,22052017,NOT FOR TRAVEL,1,S,E,...,000,000,N,N,N,87,NOT FOR TRAVEL,1,001,20
2825,RZZX311229992305201722052017NOT FOR TRAVEL 1RE...,R,ZZX,31122999,23052017,22052017,NOT FOR TRAVEL,1,R,E,...,000,000,N,N,N,81,NOT FOR TRAVEL,2,001,20
2826,RZZY311229992305201722052017NOT FOR TRAVEL 2SE...,R,ZZY,31122999,23052017,22052017,NOT FOR TRAVEL,2,S,E,...,000,000,N,N,N,87,NOT VALID FOR TRAVEL,1,001,20
